In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



    Covid had become disasterous since its genesis in China. There are around 250 million people infected with it . It castes shadows over social and economic spheres of the world. education is one of the secotors that have been hit very hard .There were global lockdowns i schools, campuses , offices , business areas etc.  Some Nations adopted online learning but countries with poor infrastructure for internet and telecom suffers the most from this crisis

    IT has disrupted learning for more than 56 million students in the United States. In the Spring of 2020, most states and local governments across the U.S. closed educational institutions to stop the spread of the virus. In response, schools and teachers have attempted to reach students remotely through distance learning tools and digital platforms. Until today, concerns of the exacaberting digital divide and long-term learning loss among America’s most vulnerable learners continue to grow

    This analysis therefore aims at analysing this common factors that bring about inequality in the education system, and through that come up with positive policies and solutions to how better not only governments but also schools can play their part in bridging the educational divide especially during this covid peri

    The following data analysis's objective is to analyse the situation the education sector is in to take the necessary shift of policies to counter the effects the pandemic in the united states. 

Datasets


the datas were collected from a competition with the following description to it 

We have provided a set of daily edtech engagement data from over 200 school districts in 2020, and we encourage you to leverage other publicly available data sources in your analysis. We include three basic sets of files to help you get started:

The engagement_ data folder is based on LearnPlatform’s Student Chrome Extension. The extension collects page load events of over 10K education technology products in our product library, including websites, apps, web apps, software programs, extensions, ebooks, hardwares, and services used in educational institutions. The engagement data have been aggregated at school district level, and each file represents data from one school district.
The products_info.csv file includes information about the characteristics of the top 372 products with most users in 2020.
The districts_info.csv file includes information about the characteristics of school districts, including data from NCES and FCC.
The definitions of each column in the three data sets are detailed in the README file.

In [ ]:
import os
import glob
import wandb
import json
import warnings
import imageio
import datetime
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML, Javascript
import IPython.display as py_display
warnings.filterwarnings("ignore")

**Utility methods** from week 1 scripts

In [ ]:
# A function for calculating the missing values in dataset in percent
def percent_missing(df: pd.DataFrame):
    # Calculate total  number of cells in dataframe
    totalCells = np.product(df.shape)

    # Count number of missing values per column
    missingCount = df.isnull().sum()

    # Calculate total number of missing values
    totalMissing = missingCount.sum()

    # Calculate percentage of missing values
    return print("The dataset contains", round(((totalMissing / totalCells) * 100), 2), "%", "missing values.")


# A function to see the percentage of missing values of every columns
def every_column_percent_missing(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_db = pd.DataFrame({'column_name': df.columns,
                                     'percent_missing': percent_missing})

    missing_value_db.sort_values('percent_missing', inplace=True)

    print(missing_value_db)
    
#PLOTING FUNCTIONS

def plot_hist(df: pd.DataFrame, column: str, color: str) -> None:
    plt.figure(figsize=(9, 7))
    sns.displot(data=df, x=column, color=color, kde=True, height=7, aspect=2)
    plt.title(f'Distribution of {column}', size=20, fontweight='bold')
    plt.show()


def plot_dist(df: pd.DataFrame, column: str):
    plt.figure(figsize=(9, 7))
    sns.distplot(df).set_title(f'Distribution of {column}')
    plt.show()


def plot_count(df: pd.DataFrame, column: str) -> None:
    plt.figure(figsize=(12, 7))
    sns.countplot(data=df, x=column)
    plt.title(f'Plot count of {column}', size=20, fontweight='bold')
    plt.show()


def plot_bar(df: pd.DataFrame, x_col: str, y_col: str, title: str, xlabel: str, ylabel: str) -> None:
    plt.figure(figsize=(9, 7))
    sns.barplot(data=df, x=x_col, y=y_col)
    plt.title(title, size=20)
    plt.xticks(rotation=75, fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.show()


def plot_heatmap(df: pd.DataFrame, title: str, cbar=False) -> None:
    plt.figure(figsize=(12, 7))
    sns.heatmap(df, annot=True, cmap='viridis', vmin=0,
                vmax=1, fmt='.2f', linewidths=.7, cbar=cbar)
    plt.title(title, size=18, fontweight='bold')
    plt.show()


def plot_box(df: pd.DataFrame, x_col: str, title: str) -> None:
    plt.figure(figsize=(12, 7))
    sns.boxplot(data=df, x=x_col)
    plt.title(title, size=20)
    plt.xticks(rotation=75, fontsize=14)
    plt.show()


def plot_box_multi(df: pd.DataFrame, x_col: str, y_col: str, title: str) -> None:
    plt.figure(figsize=(12, 7))
    sns.boxplot(data=df, x=x_col, y=y_col)
    plt.title(title, size=20)
    plt.xticks(rotation=75, fontsize=14)
    plt.yticks(fontsize=14)
    plt.show()


def plot_scatter(df: pd.DataFrame, x_col: str, y_col: str, title: str, hue: str, style: str) -> None:
    plt.figure(figsize=(10, 8))
    sns.scatterplot(data=df, x=x_col, y=y_col, hue=hue, style=style)
    plt.title(title, size=20)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.show()
    
    

In [ ]:
def bar_plot(x, y, title, palette_len, xlim = None, ylim = None, 
             xticklabels = None, yticklabels = None, 
             top_visible = False, right_visible = False, 
             bottom_visible = True, left_visible = False,
             xlabel = None, ylabel = None, figsize = (10, 4),
             axis_grid = 'y'):
    fig, ax = plt.subplots(figsize = figsize)
    plt.title(title, size = 15, fontweight = 'bold', fontfamily = 'serif')

    for i in ['top', 'right', 'bottom', 'left']:
        ax.spines[i].set_color('black')
    
    ax.spines['top'].set_visible(top_visible)
    ax.spines['right'].set_visible(right_visible)
    ax.spines['bottom'].set_visible(bottom_visible)
    ax.spines['left'].set_visible(left_visible)

    sns.barplot(x = x, y = y, edgecolor = 'black', ax = ax,
                palette = reversed(sns.color_palette("viridis", len(palette_len))))
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)    
    ax.set_xticklabels(xticklabels, fontfamily = 'serif')
    ax.set_yticklabels(yticklabels, fontfamily = 'serif')
    plt.xlabel(xlabel, fontfamily = 'serif')
    plt.ylabel(ylabel, fontfamily = 'serif')
    ax.grid(axis = axis_grid, linestyle = '--', alpha = 0.9)
    plt.show()
    
    
def line_plot(data, y, title, color,
              top_visible = False, right_visible = False, 
              bottom_visible = True, left_visible = False,
              ylabel = None, figsize = (10, 4), axis_grid = 'y'):
    fig, ax = plt.subplots(figsize = figsize)
    plt.title(title, size = 15, fontweight = 'bold', fontfamily = 'serif')

    for i in ['top', 'right', 'bottom', 'left']:
        ax.spines[i].set_color('black')
    
    ax.spines['top'].set_visible(top_visible)
    ax.spines['right'].set_visible(right_visible)
    ax.spines['bottom'].set_visible(bottom_visible)
    ax.spines['left'].set_visible(left_visible)
    
    sns.lineplot(x = range(len(data[y])), y = data[y], dashes = False, 
                 color = color, linewidth = .5)
    ax.xaxis.set_major_locator(plt.MaxNLocator(20))
    
    ax.set_xticks([])
    plt.xticks(rotation = 90)
    plt.xlabel('')
    plt.ylabel(ylabel, fontfamily = 'serif')
    ax.grid(axis = axis_grid, linestyle = '--', alpha = 0.9)
    plt.show()
    
    
def corr_plot(data,
              top_visible = False, right_visible = False, 
              bottom_visible = True, left_visible = False,
              ylabel = None, figsize = (15, 11), axis_grid = 'y'):
    fig, ax = plt.subplots(figsize = figsize)
    plt.title('Correlations (Pearson)', size = 15, fontweight = 'bold', fontfamily = 'serif')
    
    mask = np.triu(np.ones_like(data.corr(), dtype = bool))
    sns.heatmap(round(data.corr(), 2), mask = mask, cmap = 'viridis', annot = True)
    plt.show()
    
def columns_viz(data, color):
    for i in range(len(data.columns)):
        line_plot(data = data, y = data.columns[i],
                  color = color,
                  title = '{} dynamics'.format(data.columns[i]),
                  bottom_visible = False, figsize = (10, 2))
districts_info = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")
products_info = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")
eng_path = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data'


In [ ]:
# Imporrting the data and loading itto a panda daraframe 

districts_info = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")
products_info = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")


**Exploratory Data Analuysis****

looking at the distribition datas

District information data
The district file districts_info.csv includes information about the characteristics of school districts, including data from, NCES (2018-19),FCC (Dec 2018), and Edunomics Lab.

* distrist_id
* state
* locale
* pct_black/hispanic - percentage of students in the districts identified as Black or Hispanic based on 2018-19 NCES data.
* pct_free/reduced - percentage of students in the districts eligible for free or reduced-price lunch based on 2018-19 NCES data.
* county_connections_ratio - ratio (residential fixed high-speed connections over 200 kbps in at least one direction/households) based on the county level data from FCC From 477 (December 2018 version).
* pp_total_raw - per-pupil total expenditure (sum of local and federal expenditure) from Edunomics Lab's National Education Resource Database on Schools (NERD$) project.

In [ ]:
districts_info.head()

In [ ]:
districts_info.describe()

In [ ]:
# Checking for null values
districts_info.isnull().sum()

In [ ]:
percent_missing(districts_info)

In [ ]:
every_column_percent_missing(districts_info)

In [ ]:
districts_cleaned = districts_info.dropna()

In [ ]:
print(f" There are {districts_cleaned.shape[0]} rows and {districts_cleaned.shape[1]} columns")

In [ ]:
every_column_percent_missing(districts_cleaned)

In [ ]:
percent_missing(districts_cleaned)

In [ ]:
districts_cleaned.duplicated().sum()

In [ ]:
list(districts_cleaned.columns.values)

In [ ]:
plot_hist(districts_cleaned,'state','black')

In [ ]:
plot_hist(districts_cleaned,'locale','red')

In [ ]:
plot_hist(districts_cleaned,'pct_black/hispanic','green')

In [ ]:
plot_hist(districts_cleaned,'pct_free/reduced','orange')

In [ ]:
plot_hist(districts_cleaned,'pp_total_raw','green')

In [ ]:
def count_plot(data,colr,title):
    plt.figure(figsize=(10,8))
    ax=sns.countplot(x=data,palette=colr,order=data.value_counts().index)
    plt.xticks(rotation=90)
    plt.title(title)
    for p in ax.patches:
        ax.text (p.get_x() + p.get_width()  / 2,p.get_height()+ 0.75,p.get_height(), fontsize = 11)
#         ax.text('%{:.1f}'.format(p.get_height()), (p.get_x()+0.1, p.get_height()+50))


In [ ]:
count_plot(districts_cleaned['locale'],'RdYlGn','locale representation') 

In [ ]:
count_plot(districts_cleaned['state'],'RdYlGn',"State representation")

In [ ]:
count_plot(districts_cleaned['state'],'Reds',"State representation")

In [ ]:
fig = px.pie(districts_cleaned['locale'].value_counts().reset_index().rename(columns = {'locale': 'count'}), values = 'count', names = 'index', width = 700, height = 700)

fig.update_traces(textposition = 'inside', 
                  textinfo = 'percent + label', 
                  hole = 0.7, 
                  marker = dict(colors = ['#90afc5','#336b87','#2a3132','#763626'], line = dict(color = 'white', width = 2)))

fig.update_layout(annotations = [dict(text = ' The count of districts <br>in each type <br>of areas', 
                                      x = 0.5, y = 0.5, font_size = 26, showarrow = False, 
                                      font_family = 'monospace',
                                      font_color = '#283655')],
                  showlegend = False)
                  
fig.show()

**Looking at the products_cleaned**

Product information data

The product file products_info.csv includes information about the characteristics of the top 372 products with most users in 2020. The categories listed in this file are part of LearnPlatform's product taxonomy.


* LP ID - the unique identifier of the product. URL
* Product Name
* Provider/Company Name
* Sector(s) - sector of education where the product is used.
* Primary Essential Function - the basic function of the product.

* There are two layers of labels here. Products are first labeled as one of these three categories: LC = Learning & Curriculum, CM = Classroom Management, and SDO = School & District Operations. Each of these categories have multiple sub-categories with which the products were labeled

In [ ]:
products_cleaned = districts_info.dropna()

In [ ]:
list(products_cleaned.columns.values)

In [ ]:
products_info.describe()

In [ ]:
print(f" There are {products_info.shape[0]} rows and {products_info.shape[1]} columns")

In [ ]:
products_info.isnull().sum()

In [ ]:
percent_missing(products_info)

In [ ]:
every_column_percent_missing(products_info)

In [ ]:
products_cleaned = products_info.dropna()

In [ ]:
percent_missing(products_cleaned)

In [ ]:
print(f" There are {products_cleaned.shape[0]} rows and {products_cleaned.shape[1]} columns")

In [ ]:
plot_hist(products_cleaned,'Sector(s)','purple')

Engagement data¶

This particular data are based on LearnPlatform’s Student Chrome Extension. The extension collects page load events of over 10K education technology products in our product library, including websites, apps, web apps, software programs, extensions, ebooks, hardwares, and services used in educational institutions.

The engagement data are aggregated at school district level, and each file in the folder engagement_data represents data from one school district. The 4-digit file name represents district_id which can be used to link to district information in district_info.csv. The lp_id can be used to link to product information in product_info.csv.

In [ ]:
import glob
import pandas
engagement_path =("../input/learnplatform-covid19-impact-on-digital-learning/engagement_data")
engagement_merged = glob.glob(engagement_path + "/*.csv")

files = []

for file in engagement_merged:
    df = pd.read_csv(file, index_col = None, header = 0)
    district_id = file.split('/')[4].split('.')[0]
    df['district_id'] = district_id
    files.append(df)
    
merged_engagement = pd.concat(files)
merged_engagement = merged_engagement.reset_index(drop = True)
merged_engagement['time'] = pd.to_datetime(merged_engagement['time'])

In [ ]:
merged_engagement.head(10)

In [ ]:
print(f" There are {merged_engagement.shape[0]} rows and {merged_engagement.shape[1]} columns")

In [ ]:
every_column_percent_missing(merged_engagement)

In [ ]:
percent_missing(merged_engagement)

In [ ]:
merged_eng_cleaned = merged_engagement.dropna()

In [ ]:
percent_missing(merged_eng_cleaned)

In [ ]:
datasets = [districts_cleaned, products_cleaned, merged_eng_cleaned]
datasets_names = ['districts_cleaned','products_cleaned','merged_eng_cleaned']


In [ ]:
columns_viz(datasets[0], color = 'purple')

In [ ]:
for i in ['pct_black/hispanic', 'pct_free/reduced']:
    districts_cleaned[i] = districts_cleaned[i].apply(lambda x: float(x.split(',')[0][1:]) + 0.1)

districts_cleaned['pp_total_raw'] = districts_cleaned['pp_total_raw'].apply(lambda x: int(x.split(',')[0][1:]) + 1000)

districts_cleaned.drop('county_connections_ratio', axis = 1, inplace = True)

districts_cleaned.head(3)

In [ ]:
plt.figure(figsize = (15, 8))
a = sns.barplot(data = districts_cleaned['state'].value_counts().reset_index(), x = 'state', y = 'index', color = '#90afc5')
plt.xticks([])
plt.yticks(fontname = 'monospace', fontsize = 14, color = '#900C3F')
plt.title('Distribution over states',fontweight = 'bold', fontfamily = 'serif', fontsize = 20, color = '#000000')
plt.ylabel('States')
plt.xlabel('Distribution')
a.spines['left'].set_linewidth(1.5)
for w in ['right', 'top', 'bottom']:
    a.spines[w].set_visible(False)
    
for p in a.patches:
    width = p.get_width()
    plt.text(0.5 + width, p.get_y() + 0.55 * p.get_height(), f'{int(width)}',
             ha = 'center', va = 'center', fontname = 'monospace', fontsize = 15, color = '#283655')
plt.show()

In [ ]:
dist_area_group = districts_cleaned.groupby('locale').agg({'pct_black/hispanic': 'mean', 'pct_free/reduced': 'mean', 'pp_total_raw': 'mean'}).reset_index()

colors = ['#00FFFF', '#0000FF', '#808000']

fig = plt.figure(figsize = (13, 12))
for i in range(len(dist_area_group.columns.tolist()[1:])):
    plt.subplot(2, 2, i+1)
    sns.set_style("white")
    plt.title(dist_area_group.columns.tolist()[1:][i], fontweight = 'bold', fontfamily = 'serif', fontsize = 20, y = 1.09, color = colors[i])
    plt.grid(color = 'gray', linestyle = ':', axis = 'y', zorder = 0,  dashes = (1,7))
    a = sns.barplot(data = dist_area_group, x = 'locale', y = dist_area_group.columns.tolist()[1:][i], color = colors[i])
    plt.ylabel('')
    plt.xlabel('')
    plt.xticks(fontweight = 'bold', fontfamily = 'serif', fontsize = 20)
    plt.yticks([])
    
    for j in ['right', 'top', 'left']:
        a.spines[j].set_visible(False)
    for j in ['bottom']:
        a.spines[j].set_linewidth(1.4)
      
    if i < 2:
        for p in a.patches:
            height = p.get_height()
            a.annotate(f'{int(height*100)} %', (p.get_x() + p.get_width() / 2, p.get_height()-0.03), 
                   ha = 'center', va = 'center', 
                   size = 18,
                   xytext = (0, 5), 
                   textcoords = 'offset points',
                   color = 'white',
                   fontname = 'monospace')
    else:
        for p in a.patches:
            height = p.get_height()
            a.annotate(f'{int(height)} $', (p.get_x() + p.get_width() / 2, p.get_height()-1000), 
                   ha = 'center', va = 'center', 
                   size = 18,
                   xytext = (0, 5), 
                   textcoords = 'offset points',
                   color = 'white',
                   fontname = 'monospace')
            
plt.figtext(0.07, 1.05, 'Characteristics of school districts by locale', fontsize = 30, fontname = 'monospace', color = '#283655')


fig.tight_layout(pad = 3)

plt.show()

In [ ]:
plt.figure(figsize = (15, 8))
sns.set_style("white")
plt.title('TOP-15 providers', size = 35, x = 0.48, y = 1.06, fontweight = 'bold', fontfamily = 'serif', fontsize = 20, color = '#283655')
a = sns.barplot(data = products_cleaned['Provider/Company Name'].value_counts().reset_index().head(15), x = 'Provider/Company Name', y = 'index', color = '#90afc5')
plt.xticks([])
plt.yticks(fontweight = 'bold', fontfamily = 'serif', fontsize = 20, color = '#283655')
plt.ylabel('')
plt.xlabel('')

a.spines['left'].set_linewidth(1.5)
for w in ['right', 'top', 'bottom']:
    a.spines[w].set_visible(False)
#counting the numbers
for p in a.patches:
    width = p.get_width()
    plt.text(0.5 + width, p.get_y() + 0.55 * p.get_height(), f'{int(width)}',
             ha = 'center', va = 'center', fontname = 'monospace', fontsize = 15, color = '#000000')

plt.show()

In [ ]:
#Code credit Dimtry Uraov
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


state_abb = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District Of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

districts_cleaned['state_abb'] = districts_cleaned['state'].map(state_abb)

fig = go.Figure()
layout = dict(
    title_text = "Count of districts in the available States",
    title_font = dict(
            family = "monospace",
            size = 25,
            color = "black"
            ),
    geo_scope = 'usa'
)

fig.add_trace(
    go.Choropleth(
        locations = districts_cleaned['state_abb'].value_counts().to_frame().reset_index()['index'],
        zmax = 1,
        z = districts_cleaned['state_abb'].value_counts().to_frame().reset_index()['state_abb'],
        locationmode = 'USA-states',
        marker_line_color = 'white',
        geo = 'geo',
        colorscale = "cividis", 
    )
)
            
fig.update_layout(layout)   
fig.show()

checking the proportions of hispanic and black people in the districts

In [ ]:
hisp_blck_districts = districts_cleaned.groupby('state').agg({'pct_black/hispanic':np.mean,'pct_free/reduced':np.mean,'pp_total_raw':np.mean})


In [ ]:
hisp_blck_districts=hisp_blck_districts.reset_index()

In [ ]:
hisp_blck_districts['state_abb']=hisp_blck_districts['state'].map(state_abb)

In [ ]:
sns.heatmap(hisp_blck_districts.corr(),annot=True)

In [ ]:
fig = px.bar_polar(hisp_blck_districts, r="pct_black/hispanic", theta="state", 
            color_discrete_sequence= px.colors.sequential.thermal)
fig.show()

From the above plots we can see texas having the higheest black and hispanic representations by far while Newyork, New Jersey , DC and the likes. Also, there happens to be no correlation between he reduced percentsge colomns and hipanic/black represenations percentages

In [ ]:
fig = px.bar_polar(hisp_blck_districts, r="pct_free/reduced", theta="state", template="plotly_white",
            color_discrete_sequence= px.colors.sequential.Plasma_r)
fig.show()

The above plot shows the percent resuced data corresponding to the states. Minosota to be 


In [ ]:
#create a locale 
locale_df=districts_cleaned.groupby('locale').agg({'pct_black/hispanic':np.mean,'pct_free/reduced':np.mean})

In [ ]:
locale_df=locale_df.reset_index()

In [ ]:
state_locale_df=districts_cleaned.groupby(['state_abb','locale']).agg({'pct_black/hispanic':np.mean,'pct_free/reduced':np.mean,'pp_total_raw':np.mean})
state_locale_df=state_locale_df.reset_index()
state_locale_df

In [ ]:
plt.figure(figsize=(15,9))
ax=sns.barplot(x="locale", y="pct_black/hispanic", data=state_locale_df,palette="magma")

plt.xticks(rotation=90)
plt.title("Percentage of blacks and hispanic in the locales")

We can see that more hispanic and black representations are witnessed in cities and very less are in towns. 

In [ ]:
merged_data = pd.merge(products_cleaned, merged_eng_cleaned, left_on = 'LP ID', right_on = 'lp_id')
merged_data['district_id'] = merged_data['district_id'].astype('int64')
merged_data = pd.merge(merged_data, districts_cleaned, on = 'district_id')
merged_data.drop(['URL', 'lp_id', 'state_abb'], axis = 1, inplace = True)
merged_data.head(3)

In [ ]:
num_data = merged_data[['engagement_index', 'pct_black/hispanic', 'pct_free/reduced',
                                 'pct_access',  'pp_total_raw']]
data_correlation = num_data.corr()
plot_heatmap(data_correlation, 'Numerical Data Correlation')
data_correlation

    The  above correlation maps shows us there seems to be a significant positive correlation between the total page-load events per one thousand students of a given product (or on a given day) and the  percentage of students in the district which  have at least one page-load event of a given product and on a given day

    As expected , We can also observe students that are eligible for lunch or reduced-price lunches are mostly from areas where there are more hispanic and black represenataions 

In [ ]:
pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
import pandas_profiling as pp
from pandas_profiling import ProfileReport
profile = ProfileReport(merged_data,title="Pandas Profiling Report 2015", explorative=True)
profile.to_notebook_iframe()

Reference

Ruchie Bhatia

Euel Fantaye

Blaise papa

https://www.youtube.com/watch?v=AYalukmWroY


https://gist.github.com/rogerallen/1583593



In [ ]:
we 